# Import statements

In [ ]:
import numpy as np #computing
import matplotlib.pyplot as plt #plotting
import scipy.optimize as opt #optimization
import pandas as pd #data handling

# Processing images

In [ ]:
#turn the intensities into weights
#make darker spots "weigh" more
#POSSIBLE SOURCE OF SYSTEMATIC ERROR
def conv_to_weights(array):
    adj = (array-array.min())/(array.max()-array.min()) #min-max scale
    
    #apply a nonlinearity - several possible transformations
    
    return 2/(1+np.exp(3*adj)) 
    #return adj
    #return 1/(1+10*adj)
    #return (1-adj)**2

#calculate center of mass of ROI image as initial estimate for center of bead
def array_cm(array):
    ylen, xlen = array.shape
    xcoords, ycoords = np.meshgrid(range(xlen), range(ylen)) #create the coordinates from origin
    xcm = np.sum(array*xcoords)/np.sum(array)
    ycm = np.sum(array*ycoords)/np.sum(array) #calculate center of mass
    
    return xcm, ycm

# Simulate a video

In [ ]:
#returns a number of normal random samples for a given stiffness and degrees of motion
#sufficient approximation for the Ornstein-Uhlenbeck process
def gauss_OU_approx(number, expected_stiffness, ndim=2):
    conv = 1.380649*10**-23*300*10**18/269**2*1000 #conversion factor
    var_each = conv/expected_stiffness/ndim
    rands = np.random.randn(number, ndim)
    return np.sqrt(var_each)*rands

In [ ]:
#generate a perfect gaussian ring for the Gaussian optimization algorithm
#inputs: grid size, center, radius, other parameters, noise in the image or not
#outputs: gaussian ring on a grid
def gen_gaussian_ring(img_size, centerx, centery, radius, sigma=2, amp=-20, background=175, noise=0):
    values = np.arange(0, img_size)
    x_vals, y_vals = np.meshgrid(values, values) #create coordinate system
    rands = noise*np.random.randn(img_size, img_size) #some random noise
    dists = np.sqrt((x_vals-centerx)**2+(y_vals-centery)**2)-radius #calculate distance
    brightnesses = amp*np.exp(-dists**2/sigma)+background+rands #calculate and scale the gaussian distribution
    
    return brightnesses

#create a series of gaussian images as a simulated video
#inputs: number of images, other parameters
#outputs: the video (number of images x length of image x width), coordinates of exact center
def gen_gaussian_series(num_frames, img_size=40, centerx=20, centery=20, radius=9, sigma=2, amp=-20, background=175, noise=0, expected_stiffness=0.5, perfect=True):
    #set up coordinates as 3-D array (time and spatial)
    values = np.arange(0, img_size)
    x_vals, y_vals = np.meshgrid(values, values)
    
    #repeat the grid time on the first axis
    x_vals, y_vals = np.tile(x_vals, (num_frames,1,1)), np.tile(y_vals, (num_frames,1,1)) 
    
    #add movement based on the normal approx of the ornstein-uhlenbeck process
    bead_disps = gauss_OU_approx(num_frames, expected_stiffness)
    center_movex = bead_disps[:,1].reshape((-1,1,1))
    center_movey = bead_disps[:,0].reshape((-1,1,1))
    new_center_x, new_center_y = centerx+center_movex, centery+center_movey #move the centers
    
    #add camera noise
    rands = noise*np.random.randn(num_frames, img_size, img_size)
    
    #turn into images
    dists = np.sqrt((x_vals-new_center_x)**2+(y_vals-new_center_y)**2)-radius #calculate distance
    
    #make a change if we don't want a perfect Gaussian
    if not perfect:
        sigma *= 1+9*(dists<0) #drops off more slowly within ring
    
    brightnesses = amp*np.exp(-dists**2/sigma)+background+rands #calculate and scale the gaussian distribution
    
    #POSSIBLE SOURCE OF ERROR - rounding to integer values
    return brightnesses.astype('uint8'), center_movex, center_movey

# Fitting algorithms

In [ ]:
#Gaussian fit like the LabVIEW code for trapping from above
#function to be optimized by scipy module in gauss_disp()
#inputs: values to be fit, intensities, all x coords, all y coords, weights
#output: mean squared error
def fit_gauss(vals, img_array, weights=None): #fits the gaussian intensity profile to the pixels
    img_array = img_array.astype('float')
    
    #get the weights for the pixels
    if weights is None:
        weights = np.ones_like(img_array)
    weights = weights/np.sum(weights)
    
    #unpack values to be fit
    centerx, centery, radius, sigma, amp, back = vals
    
    #calculate the error
    this_gauss = gen_gaussian_ring(img_array.shape[0], centerx, centery, radius, sigma, amp, back)
    diff = img_array - this_gauss
    
    return np.sum(diff**2*weights)

#use fit_gauss() to optimize a Gaussian fit in scipy.optimize.minize()
def gauss_disp(img_array, init = None):
    img_array = img_array.astype('float')
    
    #initial estimate
    if init:
        xcm, ycm = init
    else:
        dark_weights = conv_to_weights(img_array)
        xcm, ycm = array_cm(dark_weights)
    
    #optimize the fit, starting with estimated values
    #adjust these estimates if fit does not converge
    result = opt.minimize(fit_gauss, [xcm, ycm, 9, 3, -20, 170], args=(img_array), tol=1e-8)
    best_x, best_y, best_radius, best_sig, best_amp, best_back = result.x
    
    return best_x, best_y, best_radius

In [ ]:
#CAF fit, new method from literature

#takes current array and circle, calculates the change for this iteration
#CAF_fit calls this repeatedly to perform iterations
def iter_circle(weights, xc, yc, radius, width):
    grid_size = weights.shape[0]
    
    #get points used in the calculation
    num_points = int(2*np.pi*radius)
    angles = np.arange(num_points).reshape((1,-1))/radius #length = num_points
    bottom = np.max((0, radius-width))
    radii = np.arange(bottom, radius+width+1).reshape((-1,1)) #length <= 2*width+1
    
    #turn points into weights
    #matrices with shape 2*width+1 by num_points
    x_pixels = (xc + radii*np.cos(angles)+0.5).astype('int16')
    x_pixels[x_pixels<0]=0
    x_pixels[x_pixels>grid_size-1]=grid_size-1 #set boundaries
    
    y_pixels = (yc - radii*np.sin(angles)+0.5).astype('int16')
    y_pixels[y_pixels<0]=0
    y_pixels[y_pixels>grid_size-1]=grid_size-1
    
    #fancy indexing to find the brightnesses
    intensities = weights[y_pixels, x_pixels]
    
    #calculate force
    forces = np.sum(intensities*(radii-radius), axis=0)/np.sum(intensities, axis=0)
    forces[np.isnan(forces)] = 0
    
    #calculate movement
    dx = np.sum(forces*np.cos(angles))/num_points
    dy = np.sum(forces*-np.sin(angles))/num_points
    dr = np.sum(forces)/num_points
    
    return dx, dy, dr

#use the CAF algorithm to fit an image
#inputs: array of pixels, difference between circle and annulus radii,
#minimum convergence rate, and maximum iteration count before stopping the algorithm
#output: final fit values, end convergence rate, end number of iterations
def CAF_fit(array, init = None, radius=10, width=5, min_converge = 1e-4, max_iter = 1000):    
    weights = conv_to_weights(array.astype('float')) #convert to weights
    
    #initial center
    if init:
        xc, yc = init
    else:
        xc, yc = array_cm(weights)
    
    #set up other values
    step_num = 0
    converge_rate = min_converge
    tracking = np.zeros([max_iter+1, 3])
    tracking[0,:] = xc, yc, radius
    
    #iterate CAF_fit algorithm until the fit converges
    while converge_rate >= min_converge and step_num < max_iter:
        dx, dy, dr = iter_circle(weights, xc, yc, radius, width) #calculate step
        xc, yc, radius = xc+dx, yc+dy, radius+dr #take step
        converge_rate = dx**2 + dy**2 + dr**2 #calculate rate
        step_num += 1 #track iteration number
        tracking[step_num, : ] = xc, yc, radius #track all the values  
        #print(f'{xc:.2f}, {yc:.2f}, {radius:2f}')
        
    tracking = tracking[:step_num, :] #truncate
    
    return xc, yc, radius, converge_rate, step_num

# Determine stiffness

In [ ]:
#input the centers of every fit from a video
#return variance and stiffness
def calc_stiffness(all_x, all_y, all_rad, graph=False):
    x1 = all_x - all_x.mean() #mean subtracted
    y1 = all_y - all_y.mean()
    
    conv = 1.380649*10**-23*300*10**18/269**2*1000 #conversion factor based on literature converted to sq pixels
    var_disp = np.mean(x1**2+y1**2) #variance of displacement in sq pixels
    trap_stiff = conv/var_disp #calculate the stiffness
    
    if graph: #optional to graph the data as well
        fig, (ax1, ax2, ax3) = plt.subplots(3,1, sharex = True, figsize = (4,12))

        ax1.plot(x1)
        ax1.set_ylabel('X displacements')

        ax2.plot(y1)
        ax2.set_ylabel('Y displacements')

        ax3.plot(all_rad)
        ax3.set_ylabel('Radius')
        ax3.set_xlabel('Frame number')

        plt.show()    
    
    return var_disp, trap_stiff

In [ ]:
#function that simulates trials and returns stiffness
#generates video, runs algorithm, calculates stiffness all together
def test_algorithm(fit_alg, num_frames = 100, expected_stiffness = False, noise = False, perfect=False, graph = False):
    #generate video and set up variables
    sim_video, center_movex, center_movey = gen_gaussian_series(num_frames = num_frames, img_size = 40, centerx = 20, centery = 20, sigma=2, noise=noise, expected_stiffness=expected_stiffness, perfect=perfect)
    all_x, all_y, all_rad = np.zeros((3,num_frames))
    best_x, best_y = (20,20)
    
    #based on algorithm input choice
    if fit_alg == "Gaussian":
        for ct in range(num_frames):
            best_x, best_y, best_rad = gauss_disp(sim_video[ct], init=None)
            all_x[ct] = best_x
            all_y[ct] = best_y
            all_rad[ct] = best_rad
    
    elif fit_alg == "CAF":
        for ct in range(num_frames):
            best_x, best_y, best_rad, _, _ = CAF_fit(sim_video[ct], init=(best_x, best_y))
            all_x[ct] = best_x
            all_y[ct] = best_y
            all_rad[ct] = best_rad
    
    else:
        raise ValueError("Not a valid fitting algorithm")

    var, stiffness = calc_stiffness(all_x, all_y, all_rad, graph=False)
    return var, stiffness, center_movex, center_movey, all_x, all_y, all_rad, sim_video